In [2]:
import pandas as pd
l4_links = pd.read_csv('../data/idc10_links/level_4_links.csv')

l4_links.head()

,href,text
0,/ICD10CM/Codes/A00-B99/A00-A09/A00-/A00.0,A00.0
1,/ICD10CM/Codes/A00-B99/A00-A09/A00-/A00.1,A00.1
2,/ICD10CM/Codes/A00-B99/A00-A09/A00-/A00.9,A00.9
3,/ICD10CM/Codes/A00-B99/A00-A09/A01-/A01.0,A01.0
4,/ICD10CM/Codes/A00-B99/A00-A09/A01-/A01.00,A01.00


In [3]:
train_data = pd.read_parquet('../data/processed/structured_dataset_with_discharge_summaries.train.parquet')
train_data.head()

,subject_id,hadm_id,admittime,dischtime,admission_type,discharge_location,gender,dod,age_at_admit,length_of_stay_days,...,procedures_icd10,num_procedures_total,meds_discharge_like,medication_count,policy,discharge_summary,diagnosis_codes,model,true_icd_codes,missing_codes
0,17504528,20171885,2137-06-17 20:40:00,2137-06-20 16:41:00,DIRECT EMER.,HOME,F,2142-07-24,56,2.834028,...,[0W9B30Z],1.0,"[5% Dextrose, Albuterol 0.083% Neb Soln, Aspir...",24.0,partial,"**Discharge Summary** \n**Patient:** Female, ...","[E118, Z5181, I5032, K219, D473, I10, I071, M8...",gpt-4o-mini,"[I5032, E118, I10, E785, I2510, I071, D649, I2...","[Z7901, I272, E039, D72829, F419, E785, J45909..."
1,14273001,20371042,2187-10-19 19:00:00,2187-10-21 18:13:00,OBSERVATION ADMIT,HOME,F,2188-03-27,73,1.967361,...,None,NaN,"[Acetaminophen, Aspirin, Atorvastatin, Bisacod...",19.0,all,**Discharge Summary** \n**Patient:** 73-year-...,"[Z7901, Z87891, E039, Y929, I5031, E785, K219,...",gpt-4o-mini,"[I5031, I82501, C7931, C3431, E039, Z7901, I10...",[]
2,11357031,27612249,2139-01-17 21:04:00,2139-01-22 18:00:00,OBSERVATION ADMIT,HOME HEALTH CARE,M,2144-10-28,58,4.872222,...,None,NaN,"[0.9% Sodium Chloride (Mini Bag Plus), Amoxici...",30.0,primary_only,Discharge Summary: \n\nPatient is a 58-year-ol...,[I5033],gpt-4o-mini,"[I5033, J9692, J9691, E870, E872, E662, J441, ...","[E1140, I272, E872, I129, Z6833, E11319, E662,..."
3,13673554,25741865,2176-02-25 19:39:00,2176-03-02 17:30:00,OBSERVATION ADMIT,HOME HEALTH CARE,M,2180-11-21,71,5.910417,...,[0W993ZX],1.0,"[Acetaminophen, Aspirin, Atorvastatin, Cepacol...",17.0,partial,**Patient Discharge Summary** \n**Patient Nam...,"[I129, N184, I5033, N179, Z951, C9110, E780, I...",gpt-4o-mini,"[I5033, C9110, J918, N184, N179, I129, I2510, ...",[]
4,19017808,20589756,2183-01-05 21:59:00,2183-01-08 18:35:00,OBSERVATION ADMIT,HOME HEALTH CARE,F,2184-11-16,77,2.858333,...,None,NaN,"[0.9% Sodium Chloride (Mini Bag Plus), Acetami...",25.0,partial,**Discharge Summary** \n**Patient Information...,"[N2889, K7460, I8510, I5032, I4891, N179, I878...",gpt-4o-mini,"[I5032, I8510, N179, I272, K7460, N390, I4891,...","[I272, Y929, S5001XA, W19XXXA, I10, Z85038, K7..."


In [64]:
train_data['true_icd_codes']

0       [I5032, E118, I10, E785, I2510, I071, D649, I2...
1       [I5031, I82501, C7931, C3431, E039, Z7901, I10...
2       [I5033, J9692, J9691, E870, E872, E662, J441, ...
3       [I5033, C9110, J918, N184, N179, I129, I2510, ...
4       [I5032, I8510, N179, I272, K7460, N390, I4891,...
                              ...                        
1041    [Z5111, C8339, I10, G40909, E8339, R7401, T451...
1042    [Z5112, G92, C774, C786, C7989, C772, N179, E8...
1043    [Z5111, C8378, C8290, Z86711, E119, I350, I10,...
1044    [Z5111, C8235, C8333, E876, R12, T380X5A, E877...
1045    [Z432, K50914, O8619, O9963, Z3A37, Z9049, Z86...
Name: true_icd_codes, Length: 1046, dtype: object

In [65]:
l4_links['processed_text'] =l4_links['text'].replace('\.', '', regex = True)

In [66]:
# i want to get a list of all unique icd codes in the train data
unique_codes = train_data['true_icd_codes'].explode().unique()
filtered_l4_links = l4_links[l4_links['processed_text'].isin(unique_codes)]
# dedupe the links
filtered_l4_links = filtered_l4_links.drop_duplicates(subset=['processed_text', 'href'])
print(filtered_l4_links.shape)
filtered_l4_links.head()


(2471, 3)


,href,text,processed_text
21,/ICD10CM/Codes/A00-B99/A00-A09/A02-/A02.0,A02.0,A020
23,/ICD10CM/Codes/A00-B99/A00-A09/A02-/A02.1,A02.1,A021
63,/ICD10CM/Codes/A00-B99/A00-A09/A04-/A04.4,A04.4,A044
69,/ICD10CM/Codes/A00-B99/A00-A09/A04-/A04.7,A04.7,A047
70,/ICD10CM/Codes/A00-B99/A00-A09/A04-/A04.71,A04.71,A0471


In [ ]:
# # Improved parser tailored to icd10data.com structure with span-labeled sections
# from typing import Optional, Dict, Any, List
# from bs4 import BeautifulSoup

# def parse_icd10_description(html: str) -> Dict[str, Optional[str]]:
#     soup = BeautifulSoup(html, 'html.parser')

#     def norm(text: Optional[str]) -> Optional[str]:
#         if not text:
#             return None
#         return ' '.join(text.split())

#     def join_list(items: List[str]) -> Optional[str]:
#         vals = [norm(x) for x in items if norm(x)]
#         return '\n'.join(vals) if vals else None

#     # Breadcrumbs: ol.breadcrumb2
#     breadcrumbs = None
#     bc = soup.select_one('ol.breadcrumb2')
#     if bc:
#         parts: List[str] = []
#         for li in bc.find_all('li', recursive=False):
#             a = li.find('a')
#             txt = a.get_text(' ', strip=True) if a else li.get_text(' ', strip=True)
#             if txt:
#                 parts.append(txt)
#         breadcrumbs = ' > '.join(parts) if parts else None

#     # Code (prefer schema meta[itemprop=code], fallback to span.identifierDetail)
#     code = None
#     meta_code = soup.select_one('meta[itemprop="code"]')
#     if meta_code and meta_code.has_attr('content'):
#         code = norm(meta_code['content'])
#     if not code:
#         id_span = soup.select_one('span.identifierDetail')
#         if id_span:
#             code = norm(id_span.get_text(' ', strip=True))

#     # Title and code description
#     h1 = soup.find('h1')
#     title = norm(h1.get_text(' ', strip=True)) if h1 else None
#     code_description_el = soup.select_one('h2.codeDescription')
#     code_description = norm(code_description_el.get_text(' ', strip=True)) if code_description_el else None

#     # Badges and flags
#     badges_texts: List[str] = []
#     badges = soup.select('ul#badgeList span')
#     for b in badges:
#         t = norm(b.get_text(' ', strip=True))
#         if t:
#             badges_texts.append(t)
#     badges_joined = join_list(badges_texts)
#     billable = any('billable/specific code' in (t or '').lower() for t in badges_texts)
#     poa_exempt = any('poa exempt' in (t or '').lower() for t in badges_texts)

#     # Helper to extract the first <ul>/<ol> that follows a span label (case-insensitive)
#     def extract_list_after_span(labels: List[str]) -> Optional[str]:
#         labels_norm = [' '.join(l.lower().split()) for l in labels]
#         for sp in soup.find_all('span'):
#             sp_text = ' '.join(sp.get_text(' ', strip=True).lower().split())
#             if sp_text in labels_norm:
#                 nxt = sp.find_next(['ul', 'ol'])
#                 if nxt:
#                     items = [li.get_text(' ', strip=True) for li in nxt.find_all('li', recursive=True)]
#                     return join_list(items)
#         return None

#     applicable_to = extract_list_after_span(['Applicable To'])
#     approximate_synonyms = extract_list_after_span(['Approximate Synonyms'])
#     present_on_admission = extract_list_after_span(['Present On Admission'])

#     # The first bullet list of summary under the top intro (if present)
#     summary_ul = soup.select_one('div.body-content > ul, .body-content > ul')
#     summary_bullets = None
#     if summary_ul:
#         summary_bullets = join_list([li.get_text(' ', strip=True) for li in summary_ul.find_all('li', recursive=False)])

#     # Meta description (SEO)
#     meta_desc_tag = soup.find('meta', attrs={'name': 'description'})
#     meta_description = norm(meta_desc_tag['content']) if meta_desc_tag and meta_desc_tag.has_attr('content') else None

#     return {
#         'title': title,
#         'code': code,
#         'breadcrumbs': breadcrumbs,
#         'code_description': code_description,
#         'meta_description': meta_description,
#         'badges': badges_joined,
#         'billable': billable,
#         'poa_exempt': poa_exempt,
#         'summary_bullets': summary_bullets,
#         'applicable_to': applicable_to,
#         'approximate_synonyms': approximate_synonyms,
#         'present_on_admission': present_on_admission,
#     }


In [68]:
def parse_icd10_description(html: str) -> Dict[str, Optional[str]]:
    soup = BeautifulSoup(html, 'html.parser')

    def norm(text: Optional[str]) -> Optional[str]:
        if not text:
            return None
        return ' '.join(text.split())

    def join_list(items: List[str]) -> Optional[str]:
        vals = [norm(x) for x in items if norm(x)]
        return '\n'.join(vals) if vals else None

    # Breadcrumbs: ol.breadcrumb2
    breadcrumbs = None
    bc = soup.select_one('ol.breadcrumb2')
    if bc:
        parts: List[str] = []
        for li in bc.find_all('li', recursive=False):
            a = li.find('a')
            txt = a.get_text(' ', strip=True) if a else li.get_text(' ', strip=True)
            if txt:
                parts.append(txt)
        breadcrumbs = ' > '.join(parts) if parts else None

    # Code (prefer schema meta[itemprop=code], fallback to span.identifierDetail)
    code = None
    meta_code = soup.select_one('meta[itemprop="code"]')
    if meta_code and meta_code.has_attr('content'):
        code = norm(meta_code['content'])
    if not code:
        id_span = soup.select_one('span.identifierDetail')
        if id_span:
            code = norm(id_span.get_text(' ', strip=True))

    # Title and code description
    h1 = soup.find('h1')
    title = norm(h1.get_text(' ', strip=True)) if h1 else None
    code_description_el = soup.select_one('h2.codeDescription')
    code_description = norm(code_description_el.get_text(' ', strip=True)) if code_description_el else None

    # Badges and flags
    badges_texts: List[str] = []
    badges = soup.select('ul#badgeList span')
    for b in badges:
        t = norm(b.get_text(' ', strip=True))
        if t:
            badges_texts.append(t)
    badges_joined = join_list(badges_texts)
    billable = any('billable/specific code' in (t or '').lower() for t in badges_texts)
    poa_exempt = any('poa exempt' in (t or '').lower() for t in badges_texts)

    # Helper to extract the first <ul>/<ol> that follows a span label (case-insensitive)
    def extract_list_after_span(labels: List[str]) -> Optional[str]:
        labels_norm = [' '.join(l.lower().split()) for l in labels]
        for sp in soup.find_all('span'):
            sp_text = ' '.join(sp.get_text(' ', strip=True).lower().split())
            if sp_text in labels_norm:
                nxt = sp.find_next(['ul', 'ol'])
                if nxt:
                    items = [li.get_text(' ', strip=True) for li in nxt.find_all('li', recursive=True)]
                    return join_list(items)
        return None

    applicable_to = extract_list_after_span(['Applicable To'])
    approximate_synonyms = extract_list_after_span(['Approximate Synonyms'])
    present_on_admission = extract_list_after_span(['Present On Admission'])
    clinical_information = extract_list_after_span(['Clinical Information'])  # NEW: Extract clinical information

    # The first bullet list of summary under the top intro (if present)
    summary_ul = soup.select_one('div.body-content > ul, .body-content > ul')
    summary_bullets = None
    if summary_ul:
        summary_bullets = join_list([li.get_text(' ', strip=True) for li in summary_ul.find_all('li', recursive=False)])

    # Meta description (SEO)
    meta_desc_tag = soup.find('meta', attrs={'name': 'description'})
    meta_description = norm(meta_desc_tag['content']) if meta_desc_tag and meta_desc_tag.has_attr('content') else None

    return {
        'title': title,
        'code': code,
        'breadcrumbs': breadcrumbs,
        'code_description': code_description,
        'meta_description': meta_description,
        'badges': badges_joined,
        'billable': billable,
        'poa_exempt': poa_exempt,
        'summary_bullets': summary_bullets,
        'applicable_to': applicable_to,
        'approximate_synonyms': approximate_synonyms,
        'present_on_admission': present_on_admission,
        'clinical_information': clinical_information,  # NEW: Include in return dict
    }



### scrape the icd website so that I can embed their descriptions of all the codes

In [63]:
import requests
res = requests.get('https://www.icd10data.com/ICD10CM/Codes/O00-O9A/O10-O16/O14-/O14.90', headers = DEFAULT_HEADERS)

print(res.text)

<!DOCTYPE html>
<html lang="en">
<head>

    <title>2026 ICD-10-CM Diagnosis Code O14.90: Unspecified pre-eclampsia, unspecified trimester</title>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="shortcut icon" id="lnkShortcutIcon" href="/images/icd10data_ico.webp">
    <link rel="apple-touch-icon" href="/images/icd10data_ico.webp">
    <link rel="preconnect" href="https://a.pub.network/ " crossorigin />
    <link rel="preconnect" href="https://b.pub.network/" crossorigin />
    <link rel="preconnect" href="https://c.pub.network/" crossorigin />
    <link rel="preconnect" href="https://d.pub.network/" crossorigin />
    <link rel="preconnect" href="https://c.amazon-adsystem.com" crossorigin />
    <link rel="preconnect" href="https://s.amazon-adsystem.com" crossorigin />
    <link rel="preconnect" href="https://btloader.com/" crossorigin />
    <link rel="preconne

In [69]:
parse_icd10_description(res.text)

{'title': '2026 ICD-10-CM Diagnosis Code O14.90',
 'code': 'O14.90',
 'breadcrumbs': 'ICD-10-CM Codes > O00-O9A > O10-O16 > O14- > 2026 ICD-10-CM Diagnosis Code O14.90',
 'code_description': 'Unspecified pre-eclampsia, unspecified trimester',
 'meta_description': 'ICD 10 code for Unspecified pre-eclampsia, unspecified trimester. Get free rules, notes, crosswalks, synonyms, history for ICD-10 code O14.90.',
 'badges': '2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023\n2024\n2025\n2026\nBillable/Specific Code\nMaternity Dx (12-55 years)\nFemale Dx\n2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023\n2024\n2025\n2026\nNon-Billable/Non-Specific Code',
 'billable': True,
 'poa_exempt': False,
 'summary_bullets': None,
 'applicable_to': None,
 'approximate_synonyms': 'Postpartum preeclampsia (high blood pressure and protein in urine, after childbirth)\nPostpartum preeclampsia (high blood pressure and protein in urine,after childbirth)\nPre-eclampsia\nPre-eclampsia postpartum\nPreeclampsia (high bl

## ✨ Improved Async Scraping with Checkpointing

This section implements a robust async scraper with the following features:

### Key Features:
1. **Checkpointing**: Progress is saved after every batch (50 URLs by default)
2. **Graceful 405 Handling**: When a 405 error occurs, the scraper stops immediately, saves all progress, and tells you where it failed
3. **Session Management**: Sessions are always properly closed using try/finally blocks, even when errors occur
4. **Resume Capability**: Simply re-run the scraper and it will automatically skip already-scraped URLs
5. **Batch Processing**: URLs are processed in batches to ensure regular checkpoints
6. **Error Tracking**: Each result includes an error field so you can see which URLs failed and why

### How to Use:
1. Run the scraper cell (Cell 14) - it will automatically save progress
2. If you get a 405 error, wait a while (hours/days) before resuming
3. When ready to resume, just run Cell 14 again - it loads the checkpoint automatically
4. Check progress anytime by running the checkpoint status cell (Cell 16)

### Adjustable Parameters:
- `concurrency`: Number of simultaneous requests (default: 5, lower is more polite)
- `polite_delay_s`: Delay between requests in seconds (default: 0.5)
- `batch_size`: How many URLs to scrape before saving a checkpoint (default: 50)


### Async scraping of ICD-10 code descriptions
This section replaces the legacy `urllib` approach with an asyncio + aiohttp scraper that batches requests with concurrency limits, retries, and polite delays, and parses concise descriptions from each code page.


In [70]:
import asyncio
import aiohttp
from aiohttp import ClientSession, ClientTimeout
from bs4 import BeautifulSoup
import pandas as pd
import random
import os
import json
from typing import Optional, Dict, Any, List, Set, Tuple
from tqdm.auto import tqdm
from datetime import datetime

ROOT_URL = 'https://www.icd10data.com'

DEFAULT_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) icd10-scraper/0.1',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
}

def build_full_url(href: str) -> str:
    if href.startswith('http'):
        return href
    if not href.startswith('/'):
        href = '/' + href
    return ROOT_URL + href


class Status405Error(Exception):
    """Custom exception for 405 errors to handle them specially"""
    pass


async def fetch_with_retries(session, url, *, max_attempts=3, base_delay=1.0, timeout_s=20.0):
    """Fetch a URL with retries, returns (html, error_code) tuple"""
    attempt = 0
    while attempt < max_attempts:
        attempt += 1
        try:
            async with session.get(url, timeout=aiohttp.ClientTimeout(total=timeout_s)) as resp:
                if resp.status != 200:
                    try:
                        body_preview = (await resp.text())[:400]
                    except Exception:
                        body_preview = "<non-text body>"
                    
                    error_info = {
                        'attempt': attempt,
                        'method': 'GET',
                        'requested_url': url,
                        'final_url': str(resp.url),
                        'status': resp.status,
                        'reason': resp.reason,
                        'allow': resp.headers.get('Allow'),
                        'retry_after': resp.headers.get('Retry-After'),
                        'server': resp.headers.get('Server'),
                    }
                    print(error_info)
                    
                    # Handle 405 specially - don't retry, raise immediately
                    if resp.status == 405:
                        raise Status405Error(f'405 Method Not Allowed for {url}')
                    
                    # Retry on rate limits and server errors
                    if resp.status in (429,) or 500 <= resp.status < 600:
                        retry_after = resp.headers.get('Retry-After')
                        try:
                            delay = float(retry_after) if retry_after else base_delay * (2 ** (attempt - 1))
                        except Exception:
                            delay = base_delay * (2 ** (attempt - 1))
                        await asyncio.sleep(delay)
                        continue
                    
                    # For other errors, return None
                    return None, resp.status
                
                return await resp.text(), None
                
        except Status405Error:
            # Re-raise 405 errors to be caught by the caller
            raise
        except (aiohttp.ClientError, asyncio.TimeoutError) as e:
            delay = base_delay * (2 ** (attempt - 1))
            print({'attempt': attempt, 'url': url, 'exception': type(e).__name__, 'delay': delay})
            if attempt < max_attempts:
                await asyncio.sleep(delay)
    
    return None, 'max_retries_exceeded'


def load_checkpoint(checkpoint_file: str) -> Tuple[List[Dict[str, Any]], Set[str]]:
    """Load previous results and set of completed hrefs"""
    if os.path.exists(checkpoint_file):
        df = pd.read_parquet(checkpoint_file)
        completed_hrefs = set(df['href'].dropna().tolist())
        results = df.to_dict('records')
        print(f"Loaded checkpoint: {len(results)} previously scraped results")
        return results, completed_hrefs
    return [], set()


def save_checkpoint(checkpoint_file: str, results: List[Dict[str, Any]], metadata: Dict[str, Any] = None):
    """Save results to checkpoint file"""
    if results:
        df = pd.DataFrame(results)
        df.to_parquet(checkpoint_file, index=False)
        
        # Also save metadata if provided
        if metadata:
            meta_file = checkpoint_file.replace('.parquet', '_metadata.json')
            with open(meta_file, 'w') as f:
                json.dump(metadata, f, indent=2)
        
        print(f"Checkpoint saved: {len(results)} results")


async def scrape_all(
    hrefs: List[str], 
    checkpoint_file: str = '../data/idc10_links/scrape_checkpoint.parquet',
    concurrency: int = 10, 
    polite_delay_s: float = 0.5,
    batch_size: int = 50
) -> Tuple[List[Dict[str, Any]], Optional[str]]:
    """
    Scrape all hrefs with checkpointing and graceful error handling.
    
    Returns:
        (results, error_href): results list and the href where 405 error occurred (if any)
    """
    # Load previous progress
    all_results, completed_hrefs = load_checkpoint(checkpoint_file)
    
    # Filter out already completed hrefs
    remaining_hrefs = [h for h in hrefs if h not in completed_hrefs]
    print(f"Total hrefs: {len(hrefs)}, Already completed: {len(completed_hrefs)}, Remaining: {len(remaining_hrefs)}")
    
    if not remaining_hrefs:
        print("All hrefs already scraped!")
        return all_results, None
    
    # Track where we stopped due to 405
    error_405_href = None
    session = None
    
    try:
        connector = aiohttp.TCPConnector(limit_per_host=concurrency)
        session = aiohttp.ClientSession(headers=DEFAULT_HEADERS, connector=connector)
        
        sem = asyncio.Semaphore(concurrency)
        
        async def worker(href: str) -> Dict[str, Any]:
            """Worker that handles a single URL scrape"""
            async with sem:
                url = build_full_url(href)
                
                # Add polite delay
                if polite_delay_s:
                    await asyncio.sleep(polite_delay_s + random.uniform(0, polite_delay_s / 2))
                
                try:
                    html, error_code = await fetch_with_retries(session, url)
                    
                    if html:
                        parsed = parse_icd10_description(html)
                        return {'href': href, 'url': url, 'error': None, **parsed}
                    else:
                        # Failed to fetch
                        return {
                            'href': href, 
                            'url': url, 
                            'error': str(error_code),
                            'title': None, 
                            'meta_description': None
                        }
                except Status405Error as e:
                    # Mark this specifically as a 405 error and re-raise
                    raise Status405Error(f"405 at {href}") from e
                except Exception as e:
                    # Other errors - log and return error result
                    print(f"Unexpected error scraping {href}: {type(e).__name__}: {e}")
                    return {
                        'href': href,
                        'url': url,
                        'error': f'{type(e).__name__}: {str(e)}',
                        'title': None,
                        'meta_description': None
                    }
        
        # Process in batches for better checkpointing
        for batch_start in range(0, len(remaining_hrefs), batch_size):
            batch_hrefs = remaining_hrefs[batch_start:batch_start + batch_size]
            batch_results = []
            batch_num = (batch_start // batch_size) + 1
            total_batches = (len(remaining_hrefs) + batch_size - 1) // batch_size
            
            print(f"\nProcessing batch {batch_num}/{total_batches} ({len(batch_hrefs)} URLs)")
            
            # Create tasks for this batch
            tasks = [asyncio.create_task(worker(h)) for h in batch_hrefs]
            
            # Process with progress bar
            try:
                for task in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc=f'Batch {batch_num}'):
                    try:
                        item = await task
                        batch_results.append(item)
                    except Status405Error as e:
                        # Extract href from error message
                        error_msg = str(e)
                        if "405 at " in error_msg:
                            error_405_href = error_msg.split("405 at ")[1]
                        print(f"\n⚠️  405 ERROR DETECTED at {error_405_href}")
                        print("Stopping scraper and saving progress...")
                        
                        # Cancel remaining tasks
                        for t in tasks:
                            if not t.done():
                                t.cancel()
                        
                        # Save what we have so far
                        all_results.extend(batch_results)
                        save_checkpoint(checkpoint_file, all_results, {
                            'last_run': datetime.now().isoformat(),
                            'error_405_at': error_405_href,
                            'total_scraped': len(all_results),
                            'stopped_at_batch': batch_num
                        })
                        
                        return all_results, error_405_href
                    except asyncio.CancelledError:
                        # Task was cancelled, skip it
                        pass
            
            except Exception as e:
                print(f"Batch error: {type(e).__name__}: {e}")
                # Save partial results before re-raising
                all_results.extend(batch_results)
                save_checkpoint(checkpoint_file, all_results)
                raise
            
            # Add batch results to overall results
            all_results.extend(batch_results)
            
            # Save checkpoint after each batch
            save_checkpoint(checkpoint_file, all_results, {
                'last_run': datetime.now().isoformat(),
                'total_scraped': len(all_results),
                'completed_batches': batch_num,
                'total_batches': total_batches
            })
            
    finally:
        # Always close the session properly
        if session and not session.closed:
            await session.close()
            # Give time for connections to close
            await asyncio.sleep(0.25)
    
    print(f"\n✅ Scraping complete! Total results: {len(all_results)}")
    return all_results, None
    


In [81]:
# Load level 4 links and prepare for scraping
import asyncio
import nest_asyncio
nest_asyncio.apply()

code_col = 'processed_text'
hrefs = filtered_l4_links['href'].dropna().tolist()

print(f"Total URLs to scrape: {len(hrefs)}")

# Run scraper with checkpointing
# The scraper will:
# 1. Load previous progress if it exists
# 2. Skip already scraped URLs
# 3. Save progress after each batch (50 URLs)
# 4. Stop gracefully on 405 errors and save progress
# 5. Always close the session properly

checkpoint_file = '../data/idc10_links/scrape_checkpoint.parquet'

results, error_href = await scrape_all(
    hrefs, 
    checkpoint_file=checkpoint_file,
    concurrency=5,  # Reduced from 10 to be more polite
    polite_delay_s=0.5,  # Increased delay between requests
    batch_size=50  # Save checkpoint every 50 URLs
)

print(f"\n{'='*60}")
print(f"Scraping Results:")
print(f"Total results: {len(results)}")
if error_href:
    print(f"⚠️  Stopped at 405 error for: {error_href}")
    print(f"To resume, just re-run this cell - it will skip completed URLs")
else:
    print("✅ All URLs scraped successfully!")
print(f"{'='*60}\n")

# Convert to DataFrame
out_df = pd.DataFrame(results)

# Merge with original codes
if code_col in filtered_l4_links.columns:
    out_df = out_df.merge(
        filtered_l4_links[['href', code_col]].rename(columns={code_col: 'code'}), 
        on='href', 
        how='left'
    )

# Save final results
out_dir = '../data/idc10_links'
os.makedirs(out_dir, exist_ok=True)
out_df.to_parquet(os.path.join(out_dir, 'icd10_code_descriptions.parquet'), index=False)
out_df.to_csv(os.path.join(out_dir, 'icd10_code_descriptions.csv'), index=False)

print(f"Final results saved to {out_dir}")
print(f"\nColumns in output: {list(out_df.columns)}")
print(f"Sample of results:")
out_df.head()


Total URLs to scrape: 2471
Loaded checkpoint: 2341 previously scraped results
Total hrefs: 2471, Already completed: 2341, Remaining: 130

Processing batch 1/3 (50 URLs)


Batch 1:   0%|          | 0/50 [00:00<?, ?it/s]

Checkpoint saved: 2391 results

Processing batch 2/3 (50 URLs)


Batch 2:   0%|          | 0/50 [00:00<?, ?it/s]

Checkpoint saved: 2441 results

Processing batch 3/3 (30 URLs)


Batch 3:   0%|          | 0/30 [00:00<?, ?it/s]

Checkpoint saved: 2471 results

✅ Scraping complete! Total results: 2471

Scraping Results:
Total results: 2471
✅ All URLs scraped successfully!

Final results saved to ../data/idc10_links

Columns in output: ['href', 'url', 'error', 'title', 'code_x', 'breadcrumbs', 'code_description', 'meta_description', 'badges', 'billable', 'poa_exempt', 'summary_bullets', 'applicable_to', 'approximate_synonyms', 'present_on_admission', 'clinical_information', 'code_y']
Sample of results:


,href,url,error,title,code_x,breadcrumbs,code_description,meta_description,badges,billable,poa_exempt,summary_bullets,applicable_to,approximate_synonyms,present_on_admission,clinical_information,code_y
0,/ICD10CM/Codes/A00-B99/A00-A09/A02-/A02.1,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,None,2026 ICD-10-CM Diagnosis Code A02.1,A02.1,ICD-10-CM Codes > A00-B99 > A00-A09 > A02- > 2...,Salmonella sepsis,ICD 10 code for Salmonella sepsis. Get free ru...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,NaN,Septicemia NOS,Salmonella septicemia\nSalmonella septicemia w...,None,None,A021
1,/ICD10CM/Codes/A00-B99/A00-A09/A04-/A04.7,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,None,2026 ICD-10-CM Diagnosis Code A04.7,A04.7,ICD-10-CM Codes > A00-B99 > A00-A09 > A04- > 2...,Enterocolitis due to Clostridium difficile,ICD 10 code for Enterocolitis due to Clostridi...,2016\n2017\n2018 - Converted to Parent Code\n2...,False,False,NaN,Clostridioides difficile colitis\nFoodborne in...,C difficile diarrhea\nClostridium difficile co...,None,None,A047
2,/ICD10CM/Codes/A00-B99/A00-A09/A04-/A04.4,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,None,2026 ICD-10-CM Diagnosis Code A04.4,A04.4,ICD-10-CM Codes > A00-B99 > A00-A09 > A04- > 2...,Other intestinal Escherichia coli infections,ICD 10 code for Other intestinal Escherichia c...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,NaN,Escherichia coli enteritis NOS,E coli enteritis\nIntestinal infection due to ...,None,None,A044
3,/ICD10CM/Codes/A00-B99/A00-A09/A04-/A04.71,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,None,2026 ICD-10-CM Diagnosis Code A04.71,A04.71,ICD-10-CM Codes > A00-B99 > A00-A09 > A04- > 2...,"Enterocolitis due to Clostridium difficile, re...",ICD 10 code for Enterocolitis due to Clostridi...,2018 - New Code\n2019\n2020\n2021\n2022\n2023\...,True,False,NaN,Clostridioides difficile colitis\nFoodborne in...,None,None,None,A0471
4,/ICD10CM/Codes/A00-B99/A00-A09/A02-/A02.0,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,None,2026 ICD-10-CM Diagnosis Code A02.0,A02.0,ICD-10-CM Codes > A00-B99 > A00-A09 > A02- > 2...,Salmonella enteritis,ICD 10 code for Salmonella enteritis. Get free...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,NaN,Salmonellosis,Salmonella gastroenteritis,None,Poisoning caused by ingestion of food harborin...,A020


In [82]:
out_df[out_df['clinical_information'].notnull()]

,href,url,error,title,code_x,breadcrumbs,code_description,meta_description,badges,billable,poa_exempt,summary_bullets,applicable_to,approximate_synonyms,present_on_admission,clinical_information,code_y
4,/ICD10CM/Codes/A00-B99/A00-A09/A02-/A02.0,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,None,2026 ICD-10-CM Diagnosis Code A02.0,A02.0,ICD-10-CM Codes > A00-B99 > A00-A09 > A02- > 2...,Salmonella enteritis,ICD 10 code for Salmonella enteritis. Get free...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,NaN,Salmonellosis,Salmonella gastroenteritis,None,Poisoning caused by ingestion of food harborin...,A020
20,/ICD10CM/Codes/A00-B99/A30-A49/A41-/A41.9,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,None,2026 ICD-10-CM Diagnosis Code A41.9,A41.9,ICD-10-CM Codes > A00-B99 > A30-A49 > A41- > 2...,"Sepsis, unspecified organism","ICD 10 code for Sepsis, unspecified organism. ...",2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,NaN,Septicemia NOS,Post-splenectomy septicemia\nPostprocedural se...,None,(sep-sis) the presence of bacteria or their to...,A419
25,/ICD10CM/Codes/A00-B99/A50-A64/A63-/A63.0,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,None,2026 ICD-10-CM Diagnosis Code A63.0,A63.0,ICD-10-CM Codes > A00-B99 > A50-A64 > A63- > 2...,Anogenital (venereal) warts,ICD 10 code for Anogenital (venereal) warts. G...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,NaN,Anogenital warts due to (human) papillomavirus...,"Acuminatum wart, giant\nAnal wart\nAnal warts\...",None,A raised growth on the surface of the genitals...,A630
26,/ICD10CM/Codes/A00-B99/A80-A89/A87-/A87.9,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,None,2026 ICD-10-CM Diagnosis Code A87.9,A87.9,ICD-10-CM Codes > A00-B99 > A80-A89 > A87- > 2...,"Viral meningitis, unspecified","ICD 10 code for Viral meningitis, unspecified....",2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,NaN,Arachnoiditis (spinal) NOS,Viral meningitis,None,Viral infections of the leptomeninges and suba...,A879
29,/ICD10CM/Codes/A00-B99/B00-B09/B00-/B00.9,https://www.icd10data.com/ICD10CM/Codes/A00-B9...,None,2026 ICD-10-CM Diagnosis Code B00.9,B00.9,ICD-10-CM Codes > A00-B99 > B00-B09 > B00- > 2...,"Herpesviral infection, unspecified","ICD 10 code for Herpesviral infection, unspeci...",2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,NaN,Herpes simplex infection NOS,Herpes simplex\nHerpes simplex buttock\nHerpes...,None,A group of acute infections caused by herpes s...,B009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207,/ICD10CM/Codes/Z00-Z99/Z16-Z16/Z16-/Z16.30,https://www.icd10data.com/ICD10CM/Codes/Z00-Z9...,None,2026 ICD-10-CM Diagnosis Code Z16.30,Z16.30,ICD-10-CM Codes > Z00-Z99 > Z16-Z16 > Z16- > 2...,Resistance to unspecified antimicrobial drugs,ICD 10 code for Resistance to unspecified anti...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,False,NaN,Drug resistance NOS,Infection due to multiple drug resistant micro...,None,"Diminished or failed response of an organism, ...",Z1630
2252,/ICD10CM/Codes/Z00-Z99/Z40-Z53/Z51-/Z51.81,https://www.icd10data.com/ICD10CM/Codes/Z00-Z9...,None,2026 ICD-10-CM Diagnosis Code Z51.81,Z51.81,ICD-10-CM Codes > Z00-Z99 > Z40-Z53 > Z51- > 2...,Encounter for therapeutic drug level monitoring,ICD 10 code for Encounter for therapeutic drug...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,True,NaN,Categories Z40-Z53 are intended for use to ind...,Antihypertensive agent surveillance\nMedicatio...,Z51.81 is considered exempt from POA reporting.,(fer-e-sis) a procedure in which blood is coll...,Z5181
2257,/ICD10CM/Codes/Z00-Z99/Z55-Z65/Z59-/Z59.0,https://www.icd10data.com/ICD10CM/Codes/Z00-Z9...,None,2026 ICD-10-CM Diagnosis Code Z59.0,Z59.0,ICD-10-CM Codes > Z00-Z99 > Z55-Z65 > Z59- > 2...,Homelessness,"ICD 10 code for Homelessness. Get free rules, ...",2016\n2017\n2018\n2019\n2020\n2021\n2022 - Con...,True,True,NaN,None,Homeless,None,"Persons l

### Check Checkpoint Status

Run this cell to see the status of your scraping progress:


In [22]:
# Check checkpoint status
import os
import json

checkpoint_file = '../data/idc10_links/scrape_checkpoint.parquet'
metadata_file = '../data/idc10_links/scrape_checkpoint_metadata.json'

if os.path.exists(checkpoint_file):
    checkpoint_df = pd.read_parquet(checkpoint_file)
    print(f"✅ Checkpoint exists: {len(checkpoint_df)} URLs scraped")
    
    # Show error counts
    if 'error' in checkpoint_df.columns:
        error_count = checkpoint_df['error'].notna().sum()
        print(f"   Errors encountered: {error_count}")
        if error_count > 0:
            print(f"   Error types: {checkpoint_df['error'].value_counts().to_dict()}")
    
    # Show metadata if available
    if os.path.exists(metadata_file):
        with open(metadata_file, 'r') as f:
            metadata = json.load(f)
        print(f"\n📊 Metadata:")
        for key, value in metadata.items():
            print(f"   {key}: {value}")
    
    # Calculate progress
    total_hrefs = len(filtered_l4_links['href'].dropna())
    progress_pct = (len(checkpoint_df) / total_hrefs) * 100
    print(f"\n📈 Progress: {len(checkpoint_df)}/{total_hrefs} ({progress_pct:.1f}%)")
    
else:
    print("⚠️  No checkpoint found. Run the scraper to start.")


✅ Checkpoint exists: 2471 URLs scraped
   Errors encountered: 4
   Error types: {'404': 4}

📊 Metadata:
   last_run: 2025-10-20T21:57:19.281506
   total_scraped: 2471
   completed_batches: 2
   total_batches: 2

📈 Progress: 2471/2471 (100.0%)


In [59]:
out_df = pd.read_parquet(os.path.join(out_dir, 'icd10_code_descriptions.parquet'))

In [84]:
out_df[out_df['billable'].isnull()]

,href,url,error,title,code_x,breadcrumbs,code_description,meta_description,badges,billable,poa_exempt,applicable_to,approximate_synonyms,present_on_admission,clinical_information,code_y
1977,/ICD10CM/Codes/S00-T88/T36-T50/T40-/T40.4X1A,https://www.icd10data.com/ICD10CM/Codes/S00-T8...,404,None,None,None,None,None,None,None,None,None,None,None,None,T404X1A
1979,/ICD10CM/Codes/S00-T88/T36-T50/T40-/T40.4X5A,https://www.icd10data.com/ICD10CM/Codes/S00-T8...,404,None,None,None,None,None,None,None,None,None,None,None,None,T404X5A
2050,/ICD10CM/Codes/S00-T88/T80-T88/T81-/T81.32XA,https://www.icd10data.com/ICD10CM/Codes/S00-T8...,404,None,None,None,None,None,None,None,None,None,None,None,None,T8132XA
2054,/ICD10CM/Codes/S00-T88/T80-T88/T81-/T81.4XXA,https://www.icd10data.com/ICD10CM/Codes/S00-T8...,404,None,None,None,None,None,None,None,None,None,None,None,None,T814XXA


In [85]:
out_df = out_df[out_df['billable'].notnull()]

In [86]:
out_df.to_parquet(os.path.join(out_dir, 'icd10_code_descriptions.parquet'), index=False)
out_df.to_csv(os.path.join(out_dir, 'icd10_code_descriptions.csv'), index=False)

In [54]:
out_df.tail()

,href,url,error,title,code_x,breadcrumbs,code_description,meta_description,badges,billable,poa_exempt,summary_bullets,applicable_to,approximate_synonyms,present_on_admission,code_y
2462,/ICD10CM/Codes/Z00-Z99/Z77-Z99/Z99-/Z99.11,https://www.icd10data.com/ICD10CM/Codes/Z00-Z9...,None,2026 ICD-10-CM Diagnosis Code Z99.11,Z99.11,ICD-10-CM Codes > Z00-Z99 > Z77-Z99 > Z99- > 2...,Dependence on respirator [ventilator] status,ICD 10 code for Dependence on respirator [vent...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,True,NaN,Ventilator status,Dependence on home ventilator\nDependence on r...,Z99.11 is considered exempt from POA reporting.,Z9911
2463,/ICD10CM/Codes/Z00-Z99/Z77-Z99/Z99-/Z99.2,https://www.icd10data.com/ICD10CM/Codes/Z00-Z9...,None,2026 ICD-10-CM Diagnosis Code Z99.2,Z99.2,ICD-10-CM Codes > Z00-Z99 > Z77-Z99 > Z99- > 2...,Dependence on renal dialysis,ICD 10 code for Dependence on renal dialysis. ...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,True,NaN,Hemodialysis status\nPeritoneal dialysis statu...,Acute renal failure on dialysis\nChronic kidne...,Z99.2 is considered exempt from POA reporting.,Z992
2464,/ICD10CM/Codes/Z00-Z99/Z77-Z99/Z99-/Z99.3,https://www.icd10data.com/ICD10CM/Codes/Z00-Z9...,None,2026 ICD-10-CM Diagnosis Code Z99.3,Z99.3,ICD-10-CM Codes > Z00-Z99 > Z77-Z99 > Z99- > 2...,Dependence on wheelchair,ICD 10 code for Dependence on wheelchair. Get ...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,True,NaN,Wheelchair confinement status,Dependent on wheelchair\nWheelchair bound,Z99.3 is considered exempt from POA reporting.,Z993
2465,/ICD10CM/Codes/Z00-Z99/Z77-Z99/Z98-/Z98.84,https://www.icd10data.com/ICD10CM/Codes/Z00-Z9...,None,2026 ICD-10-CM Diagnosis Code Z98.84,Z98.84,ICD-10-CM Codes > Z00-Z99 > Z77-Z99 > Z98- > 2...,Bariatric surgery status,ICD 10 code for Bariatric surgery status. Get ...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,True,NaN,Gastric banding status\nGastric bypass status ...,History of bariatric (weight loss) surgery\nHi...,Z98.84 is considered exempt from POA reporting.,Z9884
2466,/ICD10CM/Codes/Z00-Z99/Z77-Z99/Z99-/Z99.81,https://www.icd10data.com/ICD10CM/Codes/Z00-Z9...,None,2026 ICD-10-CM Diagnosis Code Z99.81,Z99.81,ICD-10-CM Codes > Z00-Z99 > Z77-Z99 > Z99- > 2...,Dependence on supplemental oxygen,ICD 10 code for Dependence on supplemental oxy...,2016\n2017\n2018\n2019\n2020\n2021\n2022\n2023...,True,True,NaN,Dependence on long-term oxygen,Dependece on nocturnal oxygen therapy\nDepende...,Z99.81 is considered exempt from POA reporting.,Z9981


In [55]:
out_df['meta_description'][2466]

'ICD 10 code for Dependence on supplemental oxygen. Get free rules, notes, crosswalks, synonyms, history for ICD-10 code Z99.81.'

In [83]:
out_df = out_df.drop(columns=['summary_bullets'])

In [61]:
# i want to strip out the leading 'ICD 10 code for <x>' from the meta_description column
out_df['meta_description'] = out_df['meta_description'].str.replace('ICD 10 code for ', '')

# remove the "Get free rules, notes, crosswalks, synonyms, history for ICD-10 code <*>" from the meta_description column. I want to match the regex for this
out_df['meta_description'] = out_df['meta_description'].str.replace(r'Get free rules, notes, crosswalks, synonyms, history for ICD-10 code [A-Z0-9\.]+\.', '', regex=True)

out_df['meta_description'] = out_df['meta_description'].str.strip()




In [62]:
out_df['meta_description'].head()

0                                Salmonella enteritis.
1          Enterocolitis due to Clostridium difficile.
2                                   Salmonella sepsis.
3    Enterocolitis due to Clostridium difficile, re...
4     Other specified bacterial intestinal infections.
Name: meta_description, dtype: object

In [37]:
train_data.columns

Index(['subject_id', 'hadm_id', 'admittime', 'dischtime', 'admission_type',
       'discharge_location', 'gender', 'dod', 'age_at_admit',
       'length_of_stay_days', 'icd10_codes', 'icd10_descriptions',
       'num_icd10_codes', 'primary_icd10', 'icd_version', 'primary_icd10_desc',
       'primary_pdgm_bucket_simple', 'procedures_icd10',
       'num_procedures_total', 'meds_discharge_like', 'medication_count',
       'policy', 'discharge_summary', 'diagnosis_codes', 'model',
       'true_icd_codes', 'missing_codes'],
      dtype='object')

In [39]:
train_data['icd10_descriptions'].head()[0]

array(['Chronic diastolic (congestive) heart failure',
       'Type 2 diabetes mellitus with unspecified complications',
       'Essential (primary) hypertension', 'Hyperlipidemia, unspecified',
       'Atherosclerotic heart disease of native coronary artery without angina pectoris',
       'Rheumatic tricuspid insufficiency', 'Anemia, unspecified',
       'Other secondary pulmonary hypertension',
       'Unspecified asthma, uncomplicated',
       'Gastro-esophageal reflux disease without esophagitis',
       'Major depressive disorder, single episode, unspecified',
       'Anxiety disorder, unspecified', 'Hypothyroidism, unspecified',
       'Other specified disorders of bone density and structure, unspecified site',
       'Paroxysmal atrial fibrillation',
       'Long term (current) use of anticoagulants',
       'Elevated white blood cell count, unspecified',
       'Essential (hemorrhagic) thrombocythemia',
       'Personal history of non-Hodgkin lymphomas',
       'Personal histo